In [1]:
def generate_class_labels(labels):

    unique_labels = list(set(labels))

    unique_labels_map = {}

    for index, label in enumerate(unique_labels, start=0):
        unique_labels_map[label] = index

    return unique_labels_map

In [2]:
VAR = {
    'max_len': 128,
    'batch_size': 32
}

In [3]:
import numpy
import pandas
import torch
import torch.nn as nn
from transformers import BertTokenizer, AdamW, BertForSequenceClassification
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

C:\Users\JS\AppData\Local\Temp\ipykernel_17132\1128236990.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


# Classification Task: Initiating Data

In [4]:
resume_df = pandas.read_csv("cleanedResumes.csv")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

labels = resume_df['Category']
label_map = generate_class_labels(labels)

resume_df['Category'] = resume_df['Category'].apply(lambda label_name: label_map[label_name])

In [5]:
import random

minimum = resume_df['Category'].value_counts().max()
current_lowest = resume_df['Category'].value_counts().min()

#Check
count = resume_df['Category'].value_counts()
remaining = 7 - count[count<minimum]

while len(remaining != 0):
    count = resume_df['Category'].value_counts()
    remaining = 7 - count[count<minimum]

    for category in remaining.index:
        someInt = random.randint(0, current_lowest-1)
        value_to_append = resume_df[
            resume_df['Category']==category
            ]['Resume'].values[someInt]


        df_to_concat = pandas.DataFrame({
            'Category': [category],
            'Resume': [value_to_append]
        })

        resume_df = pandas.concat([resume_df, df_to_concat], axis=0)

In [6]:
resume_df['Category'].value_counts()

Category
0     14
9     14
4     14
5     14
18    14
14    14
10    14
16    14
7     14
23    14
8     14
17    14
1     14
21    14
3     14
6     14
20    14
24    14
11    14
13    14
22    14
19    14
12    14
2     14
15    14
Name: count, dtype: int64

In [7]:
resumes = resume_df.Resume.values
labels = resume_df.Category.values

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_texts = [tokenizer.tokenize(resume) for resume in resumes]

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=VAR['max_len'], dtype="long", truncating="post", padding="post")

attention_masks = []
for sequence in input_ids:
    sequence_mask = [float(id > 0) for id in sequence]
    attention_masks.append(sequence_mask)

In [8]:
training_inputs, validation_test_inputs, training_labels, validation_test_labels, training_masks, validation_test_masks = train_test_split(
    input_ids, labels, attention_masks,
    random_state=42, test_size=0.3
)

validation_inputs, testing_inputs, validation_labels, testing_labels, validation_masks, testing_masks = train_test_split(
    validation_test_inputs, validation_test_labels, validation_test_masks, random_state=42, test_size=0.3
)

training_data = TensorDataset(torch.tensor(training_inputs), torch.tensor(training_masks), torch.tensor(training_labels))
training_sampler = RandomSampler(training_data)
training_dataloader = DataLoader(training_data, sampler=training_sampler, batch_size=VAR['batch_size'])

validation_data = TensorDataset(torch.tensor(validation_inputs), torch.tensor(validation_masks), torch.tensor(validation_labels))
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=VAR['batch_size'])


In [9]:
from transformers import BertConfig, BertModel
configuration = BertConfig()
model = BertModel(configuration)
configuration = model.config

from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_map.keys()))
model = nn.DataParallel(model)
model.to(device)
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, correct_bias=False)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\JS\anaconda3\envs\NLPProjectVenv\Lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [10]:
def flat_accuracy(predicted_labels, labels):
    predicted_labels = numpy.argmax(predicted_labels.to('cpu').numpy(), axis=1).flatten()
    labels = labels.to('cpu').numpy().flatten()
    return numpy.sum(predicted_labels == labels) / len(labels)


In [11]:
from tqdm import trange
epochs = 150
training_losses = []

for epoch in trange(epochs, desc="Epoch"):
    model.train()
    training_loss = 0
    training_steps = 0

    for step, batch in enumerate(training_dataloader):
        inputs = batch[0].to(device)
        attention_masks = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()
        outputs = model(inputs, attention_mask=attention_masks, labels=labels)
        # print("TRAIN", outputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        training_loss += loss.item()
        training_steps += 1

        training_losses.append(loss.item())

    average_training_loss = training_loss/training_steps
    print("Epoch {}: Average Training Loss: {}".format(epoch+1, average_training_loss))

    model.eval()
    validation_accuracy = 0
    validation_steps = 0

    for batch in validation_dataloader:
        inputs = batch[0].to(device)
        attention_masks = batch[1].to(device)
        labels = batch[2].to(device)

        with torch.no_grad():
            outputs = model(inputs, attention_mask=attention_masks, labels=labels)
            # print("VAL", outputs)

        logits = outputs.logits
        temp_validation_accuracy = flat_accuracy(logits, labels)
        validation_accuracy += temp_validation_accuracy
        validation_steps += 1

    average_validation_accuracy = validation_accuracy/validation_steps
    print("Epoch {}: Validation Accuracy: {}".format(epoch+1, average_validation_accuracy))

Epoch:   1%|          | 1/150 [00:01<04:28,  1.80s/it]

Epoch 1: Average Training Loss: 3.2773357927799225
Epoch 1: Validation Accuracy: 0.0


Epoch:   1%|▏         | 2/150 [00:03<04:21,  1.77s/it]

Epoch 2: Average Training Loss: 3.189075529575348
Epoch 2: Validation Accuracy: 0.09953703703703703


Epoch:   2%|▏         | 3/150 [00:05<04:14,  1.73s/it]

...

Epoch 148: Average Training Loss: 0.005916126538068056
Epoch 148: Validation Accuracy: 0.90625


Epoch:  99%|█████████▉| 149/150 [03:51<00:01,  1.52s/it]

Epoch 149: Average Training Loss: 0.005988648510538042
Epoch 149: Validation Accuracy: 0.90625


Epoch: 100%|██████████| 150/150 [03:52<00:00,  1.55s/it]

Epoch 150: Average Training Loss: 0.0059903645887970924
Epoch 150: Validation Accuracy: 0.90625


In [12]:
input_ids = torch.tensor(testing_inputs)
attention_masks = torch.tensor(testing_masks)
labels = torch.tensor(testing_labels)

prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_dataloader = DataLoader(prediction_data, batch_size=VAR['batch_size'])

model.eval()
logits_set = []
labels_set = []

for batch in prediction_dataloader:
    batch_input_ids, batch_attention_masks, batch_labels = batch
    batch_input_ids, batch_attention_masks, batch_labels = batch_input_ids.to(device), batch_attention_masks.to(device), batch_labels.to(device)

    with torch.no_grad():
        outputs = model(batch_input_ids, attention_mask=batch_attention_masks)
        logits = outputs.logits

    logits_set.append(logits.cpu().numpy())
    labels_set.append(batch_labels.cpu().numpy())

from sklearn.metrics import matthews_corrcoef
matthews_set = []

# Calculate Matthews correlation coefficient for each batch
for i in range(len(labels_set)):
    mcc = matthews_corrcoef(labels_set[i], numpy.argmax(logits_set[i], axis=1).flatten())
    matthews_set.append(mcc)

for i, mcc in enumerate(matthews_set):
    print(f"Batch {i + 1}: MCC = {mcc}")

# Calculate the overall Matthews correlation coefficient
overall_mcc = numpy.mean(matthews_set)
print(f"\nOverall MCC: {overall_mcc}")

Batch 1: MCC = 0.7747691715603581

Overall MCC: 0.7747691715603581
